In [7]:
import numpy as np
from typing import Dict, Union
from sturdy.validator.reward import format_allocations


def calc_similarity_matrix(apys_and_allocations: Dict[str, Dict[str, Union[Dict[str, float], float]]],
                               assets_and_pools: Dict[str, Union[Dict[str, float], float]]):
    similarity_matrix = {}
    for miner_a, info_a in apys_and_allocations.items():
        _alloc_a = info_a["allocations"]
        alloc_a = np.array(
            list(format_allocations(_alloc_a, assets_and_pools).values())
        )
        similarity_matrix[miner_a] = {}
        for miner_b, info_b in apys_and_allocations.items():
            if miner_a != miner_b:
                _alloc_b = info_b["allocations"]
                alloc_b = np.array(
                    list(format_allocations(_alloc_b, assets_and_pools).values())
                )
                similarity_matrix[miner_a][miner_b] = np.linalg.norm(alloc_a - alloc_b)
    return similarity_matrix

In [15]:
from sturdy.igreedy import igreedy_allocation_algorithm
from sturdy.pools import generate_assets_and_pools
from sturdy.protocol import AllocateAssets
from sturdy.score import calc_strategy_apy, strategies_apy
from sturdy.validator.reward import calculate_penalties


assets_and_pools = generate_assets_and_pools()
allocation = igreedy_allocation_algorithm(AllocateAssets(assets_and_pools=assets_and_pools))
apy = calc_strategy_apy(allocation, assets_and_pools)

apys_and_allocations = {
    '0': {
        'apy': apy,
        'allocations': allocation
    },
    '1': {
        'apy': apy,
        'allocations': allocation
    }
}
axon_times = {k: 0 for k, _ in apys_and_allocations.items()}

similarity_matrix = calc_similarity_matrix(apys_and_allocations, assets_and_pools)

penalties = calculate_penalties(similarity_matrix, axon_times=axon_times)

In [111]:
from scipy.spatial.distance import pdist, squareform


def generate_sphere_packing(N, M, min_dist):
    # Initialize an empty list to store the valid vectors
    vectors = []

    while len(vectors) < M:
        candidate = np.random.randn(N)  # Generate a random candidate vector
        candidate = candidate / (np.linalg.norm(candidate) * 10)  # Normalize the candidate vector
        # if all(np.linalg.norm(candidate - v) >= min_dist for v in vectors) and np.linalg.norm(candidate) < min_dist + 1e-2:
        if all(np.linalg.norm(candidate - v) >= min_dist for v in vectors):
            vectors.append(candidate)

    return np.array(vectors)

# Example parameters
N = 10  # Dimension
M = 100  # Number of vectors
min_dist = 0.1  # Minimum L2 norm distance

vectors = generate_sphere_packing(N, M, min_dist)

apys_and_allocations = {}
for idx, vec in enumerate(vectors):
    target = vec.copy() + np.array(list(allocation.values()))
    apys_and_allocations[str(idx)] = {
        'allocations': {f'{i}': target[i] for i in range(N)},
    }

axon_times = {k: 0 for k, _ in apys_and_allocations.items()}

similarity_matrix = calc_similarity_matrix(apys_and_allocations, assets_and_pools)

penalties = calculate_penalties(similarity_matrix, axon_times=axon_times)

print(penalties)

{'0': 0, '1': 0, '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '20': 0, '21': 0, '22': 0, '23': 0, '24': 0, '25': 0, '26': 0, '27': 0, '28': 0, '29': 0, '30': 0, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 0, '38': 0, '39': 0, '40': 0, '41': 0, '42': 0, '43': 0, '44': 0, '45': 0, '46': 0, '47': 0, '48': 0, '49': 0, '50': 0, '51': 0, '52': 0, '53': 0, '54': 0, '55': 0, '56': 0, '57': 0, '58': 0, '59': 0, '60': 0, '61': 0, '62': 0, '63': 0, '64': 0, '65': 0, '66': 0, '67': 0, '68': 0, '69': 0, '70': 0, '71': 0, '72': 0, '73': 0, '74': 0, '75': 0, '76': 0, '77': 0, '78': 0, '79': 0, '80': 0, '81': 0, '82': 0, '83': 0, '84': 0, '85': 0, '86': 0, '87': 0, '88': 0, '89': 0, '90': 0, '91': 0, '92': 0, '93': 0, '94': 0, '95': 0, '96': 0, '97': 0, '98': 0, '99': 0}


In [118]:
np.linalg.norm(vectors[2])
vectors[0]

array([ 0.00687023, -0.00013409,  0.00755292, -0.0612751 ,  0.05416116,
        0.04018364, -0.00178953, -0.03705576,  0.01185207, -0.00872571])